<a href="https://colab.research.google.com/github/kingseo4567/maritime-data-minig/blob/main/13%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#13주차 과제
#서울 지하철 유동인구 분석- 지도 라이브러리 Folium

import pandas as pd
import folium
from folium.plugins import HeatMap
from IPython.display import IFrame

# 1. 파일 경로
log_path = "seoul-metro-2021.logs.csv"
info_path = "seoul-metro-station-info.csv"

# 2. 데이터 로드
log_df = pd.read_csv(log_path)
station_df = pd.read_csv(info_path)

# 3. 시간 형식 변환
log_df['timestamp'] = pd.to_datetime(log_df['timestamp'], errors='coerce')

# 4. 출근/퇴근 시간 필터링
morning_df = log_df[log_df['timestamp'].dt.hour < 9]     # 출근 (9시 이전)
evening_df = log_df[log_df['timestamp'].dt.hour >= 17]   # 퇴근 (17시 이후)

# 5. 인원 집계
morning_grouped = morning_df.groupby('station_code')[['people_in', 'people_out']].sum()
evening_grouped = evening_df.groupby('station_code')[['people_in', 'people_out']].sum()

# 6. 역 정보 준비
station_info = station_df[['station.code', 'geo.latitude', 'geo.longitude']].set_index('station.code')

# 7. 병합
morning_joined = morning_grouped.join(station_info, how='inner')
evening_joined = evening_grouped.join(station_info, how='inner')

# 8. 지도 생성
morning_map = folium.Map(location=[37.55, 126.98], zoom_start=12)
evening_map = folium.Map(location=[37.55, 126.98], zoom_start=12)

# 9. 히트맵 추가 (출근: 하차, 퇴근: 승차)
HeatMap(data=morning_joined[['geo.latitude', 'geo.longitude', 'people_out']]).add_to(morning_map)
HeatMap(data=evening_joined[['geo.latitude', 'geo.longitude', 'people_in']]).add_to(evening_map)

# 10. HTML로 저장
morning_map.save("morning_commute_out.html")
evening_map.save("evening_commute_in.html")

# 11. Colab에서 동시에 보기
display(IFrame("morning_commute_out.html", width=700, height=500))
display(IFrame("evening_commute_in.html", width=700, height=500))
